In [16]:
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
import configparser
import time

In [17]:
import requests
import pandas as pd

# url = "https://nsearchives.nseindia.com/content/indices/ind_nifty500list.csv"

# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
#     "Accept": "text/csv,application/json,application/xml,text/html,*/*",
#     "Referer": "https://www.nseindia.com",
# }

# # NSE requires a session (cookies, headers)
# session = requests.Session()
# session.headers.update(headers)

# res = session.get(url, timeout=10)

# # Save locally if needed
# with open("nifty500.csv", "wb") as f:
#     f.write(res.content)

# Read into dataframe
df = pd.read_csv("data/nifty500.csv")
print(df.head())


               Company Name                Industry     Symbol Series  \
0          360 ONE WAM Ltd.      Financial Services     360ONE     EQ   
1             3M India Ltd.             Diversified    3MINDIA     EQ   
2            ABB India Ltd.           Capital Goods        ABB     EQ   
3                  ACC Ltd.  Construction Materials        ACC     EQ   
4  ACME Solar Holdings Ltd.                   Power  ACMESOLAR     EQ   

      ISIN Code  
0  INE466L01038  
1  INE470A01017  
2  INE117A01022  
3  INE012A01025  
4  INE622W01025  


In [58]:
equities = pd.read_csv("data/Equity.csv")

In [59]:
equities

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument,Sector Name,Industry New Name,Igroup Name,ISubgroup Name
0,500002,ABB India Limited,ABB,ABB India Ltd,Active,A,2.00,INE117A01022,Heavy Electrical Equipment,Equity,Industrials,Capital Goods,Electrical Equipment,Heavy Electrical Equipment
1,500003,Aegis Logistics Ltd.,AEGISLOG,Aegis Logistics Ltd,Active,A,1.00,INE208C01025,Trading - Gas,Equity,Energy,"Oil, Gas & Consumable Fuels",Gas,Trading - Gas
2,500004,Torrent Power AEC Limited,TPAEC,Torrent Power AEC Ltd,Delisted,B,10.00,INE424A01014,,Equity,-,-,-,-
3,500005,Akar Laminators Ltd.,AKARLAMIN,Akar Laminators Ltd,Delisted,XD,10.00,INE984C01013,,Equity,-,-,-,-
4,500006,Alpha Drugs India Ltd,ALPHADR,Alpha Drug India Ltd,Delisted,B,10.00,INE256B01026,,Equity,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10342,890218,SEPC LIMITED,SEPCPP,SEPC Ltd,Active,B,10.00,IN9964H01012,,Equity,-,-,-,-
10343,890219,INFIBEAM AVENUES LIMITED,INFIBPP,INFIBEAM AVENUES Ltd,Active,A,1.00,IN9483S01010,,Equity,-,-,-,-
10344,890220,KATI PATANG LIFESTYLE LIMITED,PATANGPP,KATI PATANG LIFESTYLE Ltd,Active,X,10.00,IN9237C01014,,Equity,-,-,-,-
10345,890221,Spandana Sphoorty Financial Limited,SSFLPP,Spandana Sphoorty Financial Ltd,Active,B,10.00,IN9572J01019,,Equity,-,-,-,-


In [20]:
df = pd.merge(right=equities, left=df, left_on="ISIN Code", right_on="ISIN No")[["Company Name", "Symbol", "Security Code"]]
df

,Company Name,Symbol,Security Code
0,360 ONE WAM Ltd.,360ONE,542772
1,3M India Ltd.,3MINDIA,523395
2,ABB India Ltd.,ABB,500002
3,ACC Ltd.,ACC,500410
4,ACME Solar Holdings Ltd.,ACMESOLAR,544283
...,...,...,...
500,Zee Entertainment Enterprises Ltd.,ZEEL,505537
501,Zen Technologies Ltd.,ZENTEC,533339
502,Zensar Technolgies Ltd.,ZENSARTECH,504067
503,Zydus Lifesciences Ltd.,ZYDUSLIFE,532321


In [21]:
df.to_csv("data/tracking_list.csv", index=False)

In [69]:
## Feed request params
pageno=1
strCat="Result"
strPrevDate=20250618
strScrip=500325
strSearch="P"
strToDate=20250718
strType="C"
subcategory="Financial+Results"

## PDF request params
Pname="c66ef3df-81cb-4151-b591-7ca46b8ecb23.pdf"

In [ ]:
class Tracker:
    def __init__(self, config_path: str):
        self.config = configparser.ConfigParser()
        self.config.read(config_path)
        self.data = []
        self.fetched = set()

    def getAnnouncements(self, code: int, prevDate: str = None, toDate: str = None, test: bool = False, **kwargs):
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36",
            "Accept": "application/json, text/plain, */*",
            "Referer": "https://www.bseindia.com/",
            "Origin": "https://www.bseindia.com",
            "Accept-Encoding": "gzip, deflate, br, zstd",
            "Accept-Language": "en-US,en;q=0.9"
        }
        session = requests.Session()
        session.headers.update(headers)
        
        if toDate is None:
            to_date_obj = datetime.now()
            strToDate_val = to_date_obj.strftime("%Y%m%d")
        else:
            try:
                to_date_obj = datetime.strptime(toDate, "%Y%m%d")
                strToDate_val = toDate
            except ValueError:
                print("Error parsing dates, defaulting to today")
                to_date_obj = datetime.now()
                strToDate_val = to_date_obj.strftime("%Y%m%d")

        if prevDate is None:
            prev_date_obj = to_date_obj - timedelta(days=1)
            strPrevDate_val = prev_date_obj.strftime("%Y%m%d")
        else:
            prevDate_str = str(prevDate)
            try:
                prev_date_obj = datetime.strptime(prevDate_str, "%Y%m%d")
                strPrevDate_val = prevDate_str
            except ValueError:
                prev_date_obj = to_date_obj - timedelta(days=1)
                strPrevDate_val = prev_date_obj.strftime("%Y%m%d")

        cat = kwargs.get('cat', 'Result')
        subCat = kwargs.get('subCat','Financial Results')

        params = {
            "pageno": 1,
            "strCat": cat,
            "strPrevDate": strPrevDate_val,
            "strScrip": str(code),
            "strSearch": "P",
            "strToDate": strToDate_val,
            "strType": "C",
            "subcategory": subCat
        }
        if test:
            params.pop('strScrip')
        session.params.update(params)
        response = session.get(url=self.config['URLs']['Feed'])
        try:
            self.data.extend(response.json()['Table'])
            return response.json()
        except Exception as e:
            print(e)
            return None
    
    def getProcessed(self):
        message = ""
        _data = []
        for r in self.data:
            news = r['NEWSSUB']
            fileurl = f"{self.config['URLs']['Doc']}?Pname={r['ATTACHMENTNAME']}"
            pub_time = r['DT_TM']
            id = r['NEWSID']
            _data.append({
                'id': id,
                'headline': news,
                'file_url': fileurl,
                'time': pub_time
            })
            message += f"📢 {news}\n⏰ {pub_time}\n🔗 {fileurl}\n\n"
        return message, pd.DataFrame(_data)
    
    def format_date(date: datetime):
        date_str = str(date)
        date_str = date_str.split(" ")[0].replace("-", "")
        return date_str
    
    def build_set(self, df: pd.DataFrame, prevDate: str = None, toDate: str = None):
        for i, row in df.iterrows():
            self.getAnnouncements(code=row['Security Code'], prevDate=prevDate, toDate=toDate)
            time.sleep(0.7)

In [11]:
tracker = Tracker(".ini")

In [12]:
date = datetime.today()
def format_date(date: datetime):
    date_str = str(date)
    date_str = date_str.split(" ")[0].replace("-", "")
    return date_str
format_date(date)

'20251009'

In [13]:
to_date = format_date(date)
prev_date = format_date(date - timedelta(days=2))

In [38]:
res = tracker.getAnnouncements(code=541450, prevDate='20250701', toDate='20250731', test=False, cat='Board Meeting', subCat='Outcome of Board Meeting')

In [37]:
res

{'Table': [{'NEWSID': '95fd6878-af26-4321-9d97-b73b1bcefe34',
   'SCRIP_CD': 541450,
   'XML_NAME': 'ANN_541450_95FD6878-AF26-4321-9D97-B73B1BCEFE34',
   'NEWSSUB': 'Announcement under Regulation 30 (LODR)-Newspaper Publication',
   'DT_TM': '2025-07-31T17:46:34.083',
   'NEWS_DT': '2025-07-31T17:46:34.083',
   'CRITICALNEWS': 0,
   'ANNOUNCEMENT_TYPE': 'A',
   'QUARTER_ID': None,
   'FILESTATUS': 'N    ',
   'ATTACHMENTNAME': '4826e476-373c-479c-8b07-995e78770d5b.pdf',
   'MORE': '',
   'HEADLINE': 'Newspaper Advertisement - Notice of Extra Ordinary General Meeting',
   'CATEGORYNAME': 'Company Update',
   'OLD': 1,
   'RN': 1,
   'PDFFLAG': 1,
   'NSURL': 'https://www.bseindia.com/stock-share-price/adani-green-energy-ltd/adanigreen/541450/',
   'SLONGNAME': 'Adani Green Energy Ltd',
   'AGENDA_ID': 144,
   'TotalPageCnt': 1,
   'News_submission_dt': '2025-07-31T17:46:33',
   'DissemDT': '2025-07-31T17:46:34.083',
   'TimeDiff': '00:00:01',
   'Fld_Attachsize': 5705031,
   'SUBCATNAME

In [39]:
res

{'Table': [{'NEWSID': 'd97289fb-a623-4806-acbc-7ff32999861d',
   'SCRIP_CD': 541450,
   'XML_NAME': 'ANN_541450_D97289FB-A623-4806-ACBC-7FF32999861D',
   'NEWSSUB': 'Board Meeting Outcome for Outcome Of Board Meeting Held On July 28, 2025',
   'DT_TM': '2025-07-28T14:38:06.63',
   'NEWS_DT': '2025-07-28T14:38:06.63',
   'CRITICALNEWS': 0,
   'ANNOUNCEMENT_TYPE': 'A',
   'QUARTER_ID': None,
   'FILESTATUS': 'N    ',
   'ATTACHMENTNAME': 'c89a618f-f432-48cb-8f0f-3e3f74a0f919.pdf',
   'MORE': '',
   'HEADLINE': 'Outcome of Board Meeting held on July 28, 2025',
   'CATEGORYNAME': 'Board Meeting',
   'OLD': 1,
   'RN': 1,
   'PDFFLAG': 1,
   'NSURL': 'https://www.bseindia.com/stock-share-price/adani-green-energy-ltd/adanigreen/541450/',
   'SLONGNAME': 'Adani Green Energy Ltd',
   'AGENDA_ID': 198,
   'TotalPageCnt': 1,
   'News_submission_dt': '2025-07-28T14:38:02',
   'DissemDT': '2025-07-28T14:38:06.63',
   'TimeDiff': '00:00:04',
   'Fld_Attachsize': 16641941,
   'SUBCATNAME': 'Outcome 

In [14]:
tracker.build_set(df=df[:10], prevDate='20250701', toDate='20250731')

In [15]:
_, data = tracker.getProcessed()

In [16]:
data = pd.DataFrame(data)

In [17]:
data.iloc[0]['file_url'].split("=")[1]

'178f7e5c-c294-4266-8887-647598a68350.pdf'

In [18]:
tracker.fetched.update(data['id'].to_list())

In [19]:
tracker.fetched

{'165d081d-6b45-4155-8745-fa9cd268d3e5',
 '16827028-dd1f-45ca-8d97-08685d1b06db',
 '1fe0c8bd-331d-42d2-93bd-89ccab894d5e',
 '3250e8bc-e12a-44f8-bd1c-589760bfb1f4',
 '42e27ae9-4653-4918-a084-5c9faebce655',
 '4b8758bd-2ce0-4edd-9275-5975be602fa5',
 '5a1d37c5-4c53-43bf-b0d5-0c508f42cb72',
 '6c010c6c-b3f4-4ac6-b184-92c8b9291034',
 '742c197b-e420-4386-9a3f-21da85131e10',
 '9fbf8fb5-f5a9-4cc0-a4c4-00d384474e68',
 'bc4e676e-080c-4378-80e4-60b239b04bd9',
 'd0e13c6b-c148-448f-bfd7-bd7505f05134',
 'd2978f0c-3664-4733-9bd2-daaf8fddbafc',
 'd95e6a86-d3c3-4040-abcd-427bc65d83dd',
 'dca2e2ea-7ef4-4975-9c85-891cdcaac3cb',
 'df331309-7ca6-4f45-8633-83b307adbec6',
 'e45fa46f-4256-40ba-b1c7-a386de54bae5',
 'ef7e1281-5cfa-4447-a1db-0b339c156f93'}

In [88]:
def getReport(url):
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36",
    "Accept": "application/pdf,application/octet-stream;q=0.9,*/*;q=0.8",
    "Referer": "https://www.bseindia.com/",
    "Origin": "https://www.bseindia.com",
    }

    res = requests.get(url, headers=headers, timeout=20)

    if res.status_code == 200:
        with open("result.pdf", "wb") as f:
            f.write(res.content)

In [6]:
import requests

BOT_TOKEN = tracker.config['Bot']['Token']
CHAT_ID = tracker.config['Bot']['Chat']

msg = "🚨 Test alert: Bot is working!"
url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"

res = requests.post(url, data={"chat_id": CHAT_ID, "text": msg})
print(res.json())


{'ok': True, 'result': {'message_id': 67, 'from': {'id': 8238816458, 'is_bot': True, 'first_name': 'nifty_tracker', 'username': 'nifty_listener_bot'}, 'chat': {'id': 5437323120, 'first_name': 'Faisal', 'last_name': 'Khan', 'type': 'private'}, 'date': 1757582399, 'text': '🚨 Test alert: Bot is working!'}}


In [112]:
url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendDocument"
with open("curr_numbers_data.csv", "rb") as f:
    res = requests.post(url, data={"chat_id": CHAT_ID}, files={"document": f})
print(res.json())

{'ok': True, 'result': {'message_id': 62, 'from': {'id': 8238816458, 'is_bot': True, 'first_name': 'nifty_tracker', 'username': 'nifty_listener_bot'}, 'chat': {'id': 5437323120, 'first_name': 'Faisal', 'last_name': 'Khan', 'type': 'private'}, 'date': 1757581630, 'document': {'file_name': 'curr_numbers_data.csv', 'mime_type': 'text/csv', 'file_id': 'BQACAgUAAxkDAAM-aMKRPs7SVMDBe-nPVGshmD-bxA4AAtkYAAKfPBFW68FQZ2bBjhA2BA', 'file_unique_id': 'AgAD2RgAAp88EVY', 'file_size': 367}}}


In [12]:
import pdfplumber

def extract_pages(pdf_path):
    pages_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text:
                pages_text.append((i, text))
    return pages_text


In [13]:
import re

page_filter = re.compile(r"(financial\s+highlights|results|profit|revenue|PAT|Profit After Tax|EBITDA|EPS|consilidated)", re.IGNORECASE)

def filter_financial_pages(pages_text):
    return [(i, text) for i, text in pages_text if page_filter.search(text)]


In [14]:
key = tracker.config['Keys']['OpenAI']

In [24]:
from openai import OpenAI
client = OpenAI()

def extract_with_ai(filtered_pages):
    content = "\n\n".join([f"Page {i}:\n{text}" for i, text in filtered_pages])

    prompt = f"""
        You are given excerpts from a company's financial results PDF.
        Extract the following values if present (just the number, no units/symbols) from the company's latest quarter:
        - Revenue
        - PAT (Profit After Tax)
        - EBITDA
        - EPS Basic
        - EPS Diluted

        Output JSON like:
        {{
        "Revenue": "...",
        "PAT": "...",
        "EBITDA": "...",
        "EPS Basic": "...",
        "EPS Diluted": "..."
        }}
        Text:
        {content}
    """

    resp = client.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "user", "content": prompt}]
    )
    return resp.choices[0].message.content


In [16]:
import json
import os

In [17]:
pdf_path = "PDFs//0d909eab-f472-4328-a78a-1a2cfba0df70.pdf"
pages_text = extract_pages(pdf_path)
filtered = filter_financial_pages(pages_text)

In [18]:
print(f"Actual no of pages: {len(pages_text)}, Filtered financial pages: {len(filtered)}")

Actual no of pages: 10, Filtered financial pages: 10


In [25]:
result = extract_with_ai(filtered)
print(result)

{
  "Revenue": "606.61",
  "PAT": "396.12",
  "EBITDA": "378",
  "EPS Basic": "6.24",
  "EPS Diluted": "6.13"
}


In [81]:
import pdfplumber

def detect_tabular_pages(path, min_nums_per_row=3, min_rows=5):
    pages_with_tables = []
    with pdfplumber.open(path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text() or ""
            lines = text.splitlines()
            table_like = sum(
                1 for ln in lines if sum(c.isdigit() for c in ln) >= min_nums_per_row
            )
            if table_like >= min_rows:
                pages_with_tables.append(i)
    return pages_with_tables

print(detect_tabular_pages("result.pdf"))


[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27]


In [49]:
import pandas as pd
import os

In [23]:
df = pd.read_csv("20250912.csv")

In [24]:
df = df.drop_duplicates(subset=['company'])

In [27]:
df.to_csv("Nifty_500_20250914.csv", index=False)

In [1]:
import pandas as pd

# Load skipping the control record
cols = [
    "RecordType", "Symbol", "Series", "ISIN",
    "VaR", "Filler", "VAR margin", "Extreme loss rate",
    "Ad-hoc margin", "Daily margin rate"
]

var_df = pd.read_csv("C_VAR1_15092025_2.DAT", header=None, names=cols)

print(var_df.head())


   RecordType    Symbol Series          ISIN      VaR  Filler  VAR margin  \
0          10  15092025   0.00            02  17378.0     NaN         NaN   
1          20  0MOFSL26     N1  INE338I07156      0.0     0.0        10.0   
2          20  0MOFSL27     N3  INE338I07099      0.0     0.0        10.0   
3          20   0NHIT34     N0  INE0H7R07066      0.0     0.0        10.0   
4          20   0NHIT35     N0  INE0H7R07058      0.0     0.0        10.0   

   Extreme loss rate  Ad-hoc margin  Daily margin rate  
0                NaN            NaN                NaN  
1                0.0            0.0               10.0  
2                0.0            0.0               10.0  
3                0.0            0.0               10.0  
4                0.0            0.0               10.0  


In [61]:
nifty_500 = pd.read_csv("data/nifty500.csv")
nifty_500.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
4,ACME Solar Holdings Ltd.,Power,ACMESOLAR,EQ,INE622W01025


In [3]:
mtf = pd.read_csv("Zerodha - Approved Securities for MTF.csv")
haircut = pd.read_csv("Zerodha - Approved Securities.csv")

In [4]:
mtf.head()

,isin,tradingsymbol,category,margin,leverage
0,INE117A01022,ABB,fo,23.28,4.30
1,INE208C01025,AEGISLOG,non_fo,37.32,2.68
2,INE885A01032,ARE&M,non_fo,30.79,3.25
3,INE032A01023,BOMDYEING,non_fo,36.89,2.71
4,INE100A01010,ATUL,non_fo,29.04,3.44


In [5]:
haircut.head()

,symbol,isin,haircut_value,security_name,security_segment,security_type,limit_breached
0,ABSLLIQUID,INF209KB18T9,6.0,BIRLASLAMC-ABSLLIQUID,cash,ETF,False
1,AONELIQUID,INF1J2R01056,6.0,AONEAMC - AONELIQUID,cash,ETF,False
2,CASHIETF,INF109K1A021,6.0,ICICIPRAMC - CASHIETF,cash,ETF,False
3,GROWWLIQID,INF666M01IP5,6.0,GROWWAMC - GROWWLIQID,cash,ETF,False
4,HDFCLIQUID,INF179KC1JG3,6.0,HDFCAMC-HDFCLIQUID,cash,ETF,False


In [6]:
print(f"Nifty 500 data columns: {nifty_500.columns}")
print(f"VAR data columns: {var_df.columns}")
print(f"MTF data columns: {mtf.columns}")
print(f"Haircut data columns: {haircut.columns}")

Nifty 500 data columns: Index(['Company Name', 'Industry', 'Symbol', 'Series', 'ISIN Code'], dtype='object')
VAR data columns: Index(['RecordType', 'Symbol', 'Series', 'ISIN', 'VaR', 'Filler', 'VAR margin',
       'Extreme loss rate', 'Ad-hoc margin', 'Daily margin rate'],
      dtype='object')
MTF data columns: Index(['isin', 'tradingsymbol', 'category', 'margin', 'leverage'], dtype='object')
Haircut data columns: Index(['symbol', 'isin', 'haircut_value', 'security_name', 'security_segment',
       'security_type', 'limit_breached'],
      dtype='object')


In [7]:
# --- 1. Clean column names for consistency ---
nifty = nifty_500.rename(columns={"ISIN Code": "ISIN", "Symbol": "symbol", "Series": "series"})
var = var_df.rename(columns={"VAR margin": "var_margin"})
mtf = mtf.rename(columns={"isin": "ISIN", "margin": "mtf_margin"})
haircut = haircut.rename(columns={"isin": "ISIN", "haircut_value": "haircut"})

# Ensure all ISINs uppercase & stripped
for df in [nifty, var, mtf, haircut]:
    df["ISIN"] = df["ISIN"].str.strip().str.upper()

# --- 2. Merge step by step on ISIN ---
merged = (
    nifty
    .merge(var[["ISIN", "var_margin"]], on="ISIN", how="left")
    .merge(mtf[["ISIN", "mtf_margin"]], on="ISIN", how="left")
    .merge(haircut[["ISIN", "haircut"]], on="ISIN", how="left")
)

# --- 3. Keep only required columns ---
final = merged[["Company Name", "symbol", "series", "ISIN", "var_margin", "mtf_margin", "haircut"]]

final.head()

,Company Name,symbol,series,ISIN,var_margin,mtf_margin,haircut
0,360 ONE WAM Ltd.,360ONE,EQ,INE466L01038,16.05,26.59,16.09
1,3M India Ltd.,3MINDIA,EQ,INE470A01017,9.83,27.35,9.85
2,ABB India Ltd.,ABB,EQ,INE117A01022,12.76,23.28,12.78
3,ACC Ltd.,ACC,EQ,INE012A01025,9.98,27.51,10.01
4,ACME Solar Holdings Ltd.,ACMESOLAR,EQ,INE622W01025,16.10,50.00,NaN


In [8]:
final.columns

Index(['Company Name', 'symbol', 'series', 'ISIN', 'var_margin', 'mtf_margin',
       'haircut'],
      dtype='object')

In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

cols_to_norm = ['var_margin', 'mtf_margin', 'haircut']

scaler = MinMaxScaler()
df_norm = final.copy()
df_norm[cols_to_norm] = scaler.fit_transform(final[cols_to_norm])

df_norm["risk_score"] = df_norm[cols_to_norm].mean(axis=1)

In [10]:
df_norm

,Company Name,symbol,series,ISIN,var_margin,mtf_margin,haircut,risk_score
0,360 ONE WAM Ltd.,360ONE,EQ,INE466L01038,0.077473,0.219667,0.422527,0.239889
1,3M India Ltd.,3MINDIA,EQ,INE470A01017,0.009121,0.245000,0.050656,0.101592
2,ABB India Ltd.,ABB,EQ,INE117A01022,0.041319,0.109333,0.225268,0.125307
3,ACC Ltd.,ACC,EQ,INE012A01025,0.010769,0.250333,0.060191,0.107098
4,ACME Solar Holdings Ltd.,ACMESOLAR,EQ,INE622W01025,0.078022,1.000000,NaN,0.539011
...,...,...,...,...,...,...,...,...
496,Zee Entertainment Enterprises Ltd.,ZEEL,EQ,INE256A01028,0.116703,0.572667,0.636472,0.441947
497,Zen Technologies Ltd.,ZENTEC,EQ,INE251B01027,0.140549,1.000000,0.765197,0.635249
498,Zensar Technolgies Ltd.,ZENSARTECH,EQ,INE520A01027,0.072857,0.438667,0.396901,0.302808
499,Zydus Lifesciences Ltd.,ZYDUSLIFE,EQ,INE010B01027,0.009670,0.013333,0.053635,0.025546


In [11]:
df_norm.to_csv("Nifty_500_Risk_Score.csv", index=False)

In [11]:
final.to_csv("var_mtf_data.csv", index=False)

In [33]:
df.head()

,company,date,Revenue,PAT,EBITDA,EPS Basic,EPS Diluted,file_url
0,360 ONE WAM LTD,2025-07-17T16:31:52.283,980.04,284.67,NaN,7.16,6.95,https://www.bseindia.com/stockinfo/AnnPdfOpen....
1,3M India Ltd,2025-08-07T13:15:55.247,11959687,1776864,25700.0,157.73,157.73,https://www.bseindia.com/stockinfo/AnnPdfOpen....
2,Aadhar Housing Finance Ltd,2025-07-25T16:25:16.603,84818,23728,NaN,5.50,5.37,https://www.bseindia.com/stockinfo/AnnPdfOpen....
3,Aarti Industries Ltd,2025-07-31T17:55:11.04,1676,43,NaN,1.19,1.19,https://www.bseindia.com/stockinfo/AnnPdfOpen....
4,AAVAS Financiers Ltd,2025-08-12T16:46:14.237,62756.11,13923.34,NaN,17.59,17.45,https://www.bseindia.com/stockinfo/AnnPdfOpen....


In [34]:
nifty_500.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
4,ACME Solar Holdings Ltd.,Power,ACMESOLAR,EQ,INE622W01025


In [42]:
nifty_500_companies = set([c.replace(".", "").lower() for c in nifty_500['Company Name'].to_list()])
extracted_companies = set([c.lower() for c in df['company'].to_list()])

In [46]:
nifty_500_companies.difference(extracted_companies)

{'ajanta pharmaceuticals ltd',
 'alkyl amines chemicals ltd',
 'anant raj ltd',
 'astrazenca pharma india ltd',
 'aurobindo pharma ltd',
 'balkrishna industries ltd',
 'bandhan bank ltd',
 'beml ltd',
 'bharti airtel ltd',
 'blue star ltd',
 'bse ltd',
 'castrol india ltd',
 'ccl products (i) ltd',
 'ce info systems ltd',
 'central depository services (india) ltd',
 'century plyboards (india) ltd',
 'cg power and industrial solutions ltd',
 'chambal fertilizers & chemicals ltd',
 'chennai petroleum corporation ltd',
 'coforge ltd',
 'crisil ltd',
 'cyient ltd',
 'deepak fertilisers & petrochemicals corp ltd',
 'deepak nitrite ltd',
 "divi's laboratories ltd",
 'dr lal path labs ltd',
 "dr reddy's laboratories ltd",
 'dummy valor estate ltd',
 'eid parry (india) ltd',
 'elecon engineering co ltd',
 'emami ltd',
 'endurance technologies ltd',
 'fertilisers and chemicals travancore ltd',
 'garden reach shipbuilders & engineers ltd',
 'ge vernova t&d india ltd',
 'godawari power & ispat lt

In [48]:
len(extracted_companies.difference(nifty_500_companies))

35

In [ ]:
import requests

url = "https://www.archive.nseclearing.in/content/allreports/cm/C_VAR1_15092025_2.DAT"

headers = {
    "accept": "application/json, text/plain, */*",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en-US,en;q=0.9,de-US;q=0.8,de;q=0.7",
    "cache-control": "no-cache",
    "connection": "keep-alive",
    "cookie": "auth_token=your_auth_token; session_id=your_session_id",
    "host": "www.archive.nseclearing.in",
    "pragma": "no-cache",
    "referer": "https://www.nseclearing.in/",
    "sec-ch-ua": '"Chromium";v="140", "Not=A?Brand";v="24", "Google Chrome";v="140"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/140.0.0.0 Safari/537.36",
}

resp = requests.get(url, headers=headers, timeout=120)

if resp.status_code == 200:
    with open("C_VAR1_15092025_2_python.DAT", "wb") as f:
        f.write(resp.content)
    print("Downloaded successfully:", len(resp.content), "bytes")
else:
    print("Failed:", resp.status_code, resp.text)


Downloaded successfully: 1025259 bytes


In [ ]:
from datetime import datetime, timedelta

# Last 10 quarter end dates (most recent first, ending 30-Jun-2025)
quarter_ends = [
    # "2025-06-30",
    "2025-03-31",
    "2024-12-31",
    "2024-09-30",
    "2024-06-30",
    "2024-03-31",
    "2023-12-31",
    "2023-09-30",
    # "2023-06-30",
    # "2023-03-31"
]

# Convert to datetime
quarter_ends = [datetime.strptime(d, "%Y-%m-%d") for d in quarter_ends]

# Compute upload windows (45 days after quarter end)
upload_windows = []
for q_end in quarter_ends:
    start = q_end + timedelta(days=1)
    end = start + timedelta(days=44)   # 45 days inclusive
    upload_windows.append((q_end.date(), start.date(), end.date()))

# Print nicely
for q_end, start, end in upload_windows:
    print(f"Quarter End: {q_end} | Upload Window: {start} → {end}")


Quarter End: 2025-06-30 | Upload Window: 2025-07-01 → 2025-08-14
Quarter End: 2025-03-31 | Upload Window: 2025-04-01 → 2025-05-15
Quarter End: 2024-12-31 | Upload Window: 2025-01-01 → 2025-02-14
Quarter End: 2024-09-30 | Upload Window: 2024-10-01 → 2024-11-14
Quarter End: 2024-06-30 | Upload Window: 2024-07-01 → 2024-08-14
Quarter End: 2024-03-31 | Upload Window: 2024-04-01 → 2024-05-15
Quarter End: 2023-12-31 | Upload Window: 2024-01-01 → 2024-02-14
Quarter End: 2023-09-30 | Upload Window: 2023-10-01 → 2023-11-14
Quarter End: 2023-06-30 | Upload Window: 2023-07-01 → 2023-08-14
Quarter End: 2023-03-31 | Upload Window: 2023-04-01 → 2023-05-15


In [3]:
start, end

(datetime.date(2023, 4, 1), datetime.date(2023, 5, 15))

In [40]:
import pandas as pd
import requests
import json

In [62]:
nse_100 = pd.read_csv("data/nifty500.csv")

In [63]:
with open("eodhd_nse_list.json", 'r') as f:
    eodhd_list = json.load(f)

In [64]:
eodhd = pd.DataFrame(eodhd_list)
eodhd

,Code,Name,Country,Exchange,Currency,Type,Isin
0,0P00009J3K,HDFC Mid-cap Opportunities Fund Growth,India,NSE,INR,FUND,None
1,0P0000XV6P,Kotak Large & Midcap Direct Growth,India,NSE,INR,FUND,INF174K01LF9
2,0P0000XVOU,Tata Ethical Fund Direct Plan Growth,India,NSE,INR,FUND,INF277K01NG4
3,0P0000XVPU,Taurus Ethical Fund-direct Plan Growth,India,NSE,INR,FUND,INF044D01CJ0
4,0P0000XW0O,Canara Robeco Large Cap Fund Direct Plan Growth,India,NSE,INR,FUND,INF760K01FR2
...,...,...,...,...,...,...,...
2646,ZOTA,Zota Health Care LImited,India,NSE,INR,Common Stock,INE358U01012
2647,ZUARI,Zuari Agro Chemicals Limited,India,NSE,INR,Common Stock,INE840M01016
2648,ZUARIIND,ZUARI INDUSTRIES LIMITED,India,NSE,INR,Common Stock,INE217A01012
2649,ZYDUSLIFE,Zydus Lifesciences Limited,India,NSE,INR,Common Stock,INE010B01027


In [65]:
nse_100

,Company Name,Industry,Symbol,Series,ISIN Code
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
4,ACME Solar Holdings Ltd.,Power,ACMESOLAR,EQ,INE622W01025
...,...,...,...,...,...
496,Zee Entertainment Enterprises Ltd.,Media Entertainment & Publication,ZEEL,EQ,INE256A01028
497,Zen Technologies Ltd.,Capital Goods,ZENTEC,EQ,INE251B01027
498,Zensar Technolgies Ltd.,Information Technology,ZENSARTECH,EQ,INE520A01027
499,Zydus Lifesciences Ltd.,Healthcare,ZYDUSLIFE,EQ,INE010B01027


In [66]:
nifty_500 = pd.merge(eodhd, nse_100, left_on='Code', right_on='Symbol', how='inner')

In [69]:
nifty_500 = pd.merge(nifty_500, equities, left_on='ISIN Code', right_on='ISIN No', how='inner')

In [74]:
nifty_500.to_csv("data/EODHD_Codes.csv", index=False)

In [14]:
def fetch_hourly(code):
    url = f"https://eodhd.com/api/intraday/{code}.NSE?api_token=68be768b173124.42875509&fmt=json&interval=1h&from=1577836800&to=1758385631"
    res = requests.get(url)
    data = res.json()
    return pd.DataFrame(data)

In [ ]:
import time
from tqdm import tqdm
for _, row in tqdm(nifty_500.iterrows(), desc="Processing"):
    code = row['Code']
    d = fetch_hourly(code)
    d.to_csv(f"hourly_data/{code}.csv", index=False)
    time.sleep(0.2)

Processing: 0it [00:00, ?it/s]

Processing: 100it [04:52,  2.93s/it]


In [35]:
import requests
import pandas as pd
def fetch_from_nse(symbol, from_date, to_date):
    url = f"https://www.nseindia.com/api/historicalOR/generateSecurityWiseHistoricalData?from={from_date}&to={to_date}&symbol={symbol}&type=priceVolume&series=ALL"
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36",
    "Accept": "application/pdf,application/octet-stream;q=0.9,*/*;q=0.8",
    "Referer": "https://www.nseindia.com/",
    "Origin": "https://www.nseindia.com",
    }
    res = requests.get(url, headers=headers, timeout=20)
    return pd.DataFrame(res.json()['data'])

In [57]:
int(pd.to_datetime("2025-10-07").timestamp())

1759795200

In [38]:
import time
from tqdm import tqdm
for _, row in tqdm(nifty_100.iterrows(), desc="Processing"):
    code = row['Code']
    data = pd.DataFrame()
    for date in pd.date_range(start='2019-01-01', end='2025-09-24', freq='Y'):
        from_date = date
        to_date = date+pd.Timedelta(days=365)
        d = fetch_from_nse(code, from_date.strftime(format='%d-%m-%y'), to_date.strftime(format='%d-%m-%y'))
        data = pd.concat([data, d], axis=0)
    data = data.rename(columns={"CH_SYMBOL": "symbol", "mTIMESTAMP": 'date', 'CH_OPENING_PRICE': 'open', 'CH_TRADE_HIGH_PRICE': 'high', 'CH_TRADE_LOW_PRICE': 'low', 'CH_CLOSING_PRICE': 'close',
                          'CH_TOT_TRADED_QTY': 'volume'})
    data.to_csv(f"daily_data_nse/{code}.csv", index=False)
    time.sleep(0.2)

Processing: 2it [01:36, 48.29s/it]


KeyboardInterrupt: 

In [34]:
for date in pd.date_range(start='2019-01-01', end='2025-09-24', freq='Y'):
    from_date = date
    to_date = date+pd.Timedelta(days=365)
    print(from_date, to_date)

2019-12-31 00:00:00 2020-12-30 00:00:00
2020-12-31 00:00:00 2021-12-31 00:00:00
2021-12-31 00:00:00 2022-12-31 00:00:00
2022-12-31 00:00:00 2023-12-31 00:00:00
2023-12-31 00:00:00 2024-12-30 00:00:00
2024-12-31 00:00:00 2025-12-31 00:00:00


In [26]:
date.strftime(format='%d-%m-%y')

'31-12-24'